In [2]:
import numpy as np
import pandas as pd
# import string
# import nltk
from nltk.corpus import stopwords
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
# from wordcloud import WordCloud
# from textblob import Word

# import matplotlib.pyplot as plt

In [ ]:
technical_skills = [
    'python',
    'flask',
    'pandas',
    'spark',
    'scikit-learn',
    'numpy',
    'sql',
    'mysql',
    'nltk',
    'fastapi',
    'pytorch',
    'snowflake',
    'pandas',
    'rivery',
    'django',
    'react',
    'html',
    'machine learning',
    'tableau',
    'bi',
    'powerbi',
    'looker',
    'data warehouse',
    'dbt',
    'power bi',
    'ia',
    'ai',
    'dataiku',
    'data iku',
    'r',
    'datalake',
    'data lake',
    'data',
    'ml',
    'scala',
    'api',
    'aws'
]

In [8]:
def prepare_texts(website):
    stop = stopwords.words("french")
    if website == "wttj":
        df = pd.read_csv("datasets/WTTJ_offers.csv")
    if website == "pole emploi":
        df = pd.read_csv("datasets/pole_emploi_offers.csv")
    df = df[["intitule", "description"]]
    df["description"] = df["description"].apply(
        lambda x: " "
        .join(x.lower() for x in x.split())
    ).str.replace('[^\w\s]', ' ', regex=True).apply(lambda x: ' '.join(
        x for x in x.split() if x not in stop)
    )
    df["intitule"] = df["intitule"].apply(
        lambda x: " ".join(x.lower() for x in x.split())
    ).str.replace('[^\w\s]', ' ', regex=True).apply(lambda x: ' '.join(
        x for x in x.split() if x not in stop)
    ).str.replace('h', '').str.replace('f', '')
    return df


def extract_tech_skills(description):
    skills = set(description.lower().split())
    extracted_skills = list(skills.intersection(technical_skills))
    return extracted_skills

In [12]:
df = prepare_texts("pole emploi")
df["tech_skills"] = df["description"].apply(extract_tech_skills)
df

,intitule,description,tech_skills
0,responsable data analyst,recherchons manager responsable data analyste ...,[data]
1,data analyst,aussi pensez tout mesurable passionné monde da...,[data]
2,data analyst,recherchons client spécialisé collecte tri déc...,[data]
3,data analyst expérimenté,chez k lagan parions recherche projet stable d...,"[spark, python, data]"
4,data business analyst,e data business analyst confirmé e senior doma...,[data]
...,...,...,...
281,responsable équipe risk analytics,recherchons e data analyste transformation dig...,"[api, sql, python, powerbi, data]"
282,analyste qualité données,poste missions rattaché e direction pilotage p...,"[sql, bi, data, tableau]"
283,tec lead,contexte proposons opportunité unique directio...,[api]
284,analyste bi,responsabilités missions intégré équipes proje...,[bi]


# TESTS


In [3]:
# df = pd.read_csv("WTT_offers.csv")

# df = df[["name", "description"]]
# df

NameError: name 'pd' is not defined

In [4]:
# df["description"] = df["description"].apply(lambda x: " ".join(x.lower() for x in x.split()))
# df["description"] = df["description"].str.replace('[^\w\s]', ' ')
# # df["description"] = df["description"].str.replace('\d+', '')
# stop = stopwords.words("french")
# df["description"] = df["description"].apply(lambda x: ' '.join(x for x in x.split() if x not in stop))
# df["description"] = df["description"].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

# df.head()

C:\Users\royde\AppData\Local\Temp\ipykernel_7160\1772313848.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df["description"] = df["description"].str.replace('[^\w\s]', ' ')


,name,description
0,Manager - Data & Analytics Engineering,the data team at welcome to the jungle part of...
1,Margo Analytics - Data Engineer - H/F,margo analytics entité experte margo group pro...
2,Data Analyst,être data analyst chez wewyse intégrer communa...
3,Data Analyst - Stage de 6 mois,afin mieux comprendre client leurs besoins aus...
4,Data Analyst - Stage - Paris,travailler chez papernest définition certains ...


In [5]:
# df["name"] = df["name"].apply(lambda x: " ".join(x.lower() for x in x.split()))
# df["name"] = df["name"].str.replace('[^\w\s]', ' ')
# stop = stopwords.words("french")
# df["name"] = df["name"].apply(lambda x: ' '.join(x for x in x.split() if x not in stop))
# df["name"] = df["name"].apply(lambda x: ' '.join([Word(word).lemmatize() for word in x.split()]))
# df["name"] = df["name"].str.replace('h', '').str.replace('f', '')
# df.head()

C:\Users\royde\AppData\Local\Temp\ipykernel_7160\3600074061.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df["name"] = df["name"].str.replace('[^\w\s]', ' ')


,name,description
0,manager data analytics engineering,the data team at welcome to the jungle part of...
1,margo analytics data engineer,margo analytics entité experte margo group pro...
2,data analyst,être data analyst chez wewyse intégrer communa...
3,data analyst stage 6 mois,afin mieux comprendre client leurs besoins aus...
4,data analyst stage paris,travailler chez papernest définition certains ...


In [6]:
# df["name"][df["name"].str.contains("engineer")] = "Data Engineer"
# df["name"][df["name"].str.contains("analyst")] = "Data Analyst"
# df["name"][df["name"].str.contains("scientist")] = "Data Scientist"